In [5]:
import pandas as pd
import polars as pl
import glob
import csv
import os
import pathlib

### Inputs

In [2]:
input_path = pathlib.Path(r'C:\Users\Casal-Inked\Documents\Jupyter_Lab\Dataset\ANAC')
output_path = pathlib.Path(r'C:\Users\Casal-Inked\Documents\Jupyter_Lab\Dataset\ANAC\processed')
output_path.mkdir(parents=True, exist_ok=True)

In [3]:
glob_path = str(input_path / '*.txt')
txt_files = glob.glob(glob_path)

if not txt_files:
    print(f"Nenhum arquivo .txt encontrado em '{input_path}'")
else:
    print(f"Encontrados {len(txt_files)} arquivo(s) .txt para converter\n")

# Processar cada arquivo
for txt_file in txt_files:
    try:
        base_name = os.path.basename(txt_file)
        output_file = output_path / f"{os.path.splitext(base_name)[0]}.parquet"
        
        print(f"Processando: {base_name}")
        
        # Tentar diferentes encodings comuns em arquivos brasileiros
        encodings = [# UTF-8 - Padrão moderno e mais comum
                    'utf-8',
                    'utf-8-sig',  # UTF-8 com BOM (Byte Order Mark)
                    # ISO 8859-1 (Latin-1) - Muito comum em arquivos antigos brasileiros
                    'iso-8859-1',
                    'latin-1',
                    'latin1',
                    # Windows Code Pages - Usado por sistemas Windows brasileiros
                    'cp1252',      # Windows-1252 (Western European)
                    'windows-1252',
                    'cp850',       # MS-DOS Latin-1
                    'cp437',       # MS-DOS US
                    # ISO 8859-15 - Variante do ISO-8859-1 com símbolo do Euro
                    'iso-8859-15',
                    'latin-9',
                    # Outras variantes ISO para português
                    'iso-8859-2',  # Central European
                    'iso-8859-3',  # South European
                    # MacRoman - Usado em Macs antigos
                    'mac_roman',
                    'macroman',
                    # Encodings específicos do Brasil
                    'cp860',       # MS-DOS Portuguese
                    # UTF-16 e UTF-32 (menos comuns mas possíveis)
                    'utf-16',
                    'utf-16-le',
                    'utf-16-be',
                    'utf-32',
                    # ASCII (subset de UTF-8, mas pode ajudar)
                    'ascii',]
        df = None
        
        for encoding in encodings:
            try:
                # sep=None permite que o pandas detecte automaticamente o delimitador
                df = pd.read_csv(txt_file, sep=None, engine='python', encoding=encoding)
                print(f"  ✓ Lido com sucesso (encoding: {encoding})")
                print(f"  ✓ Dimensões: {float(df.shape[0])} linhas x {float(df.shape[1])} colunas")
                break
            except (UnicodeDecodeError, Exception) as e:
                if encoding == encodings[-1]:
                    raise Exception(f"Não foi possível ler o arquivo com nenhum encoding testado")
                continue
        
        # Salvar como parquet
        df.to_parquet(output_file, index=False, compression='snappy', engine='pyarrow')
        print(f"  ✓ Convertido para: {output_file}\n")
        
    except Exception as e:
        print(f"  ✗ Erro ao processar '{txt_file}': {str(e)}\n")

print("Conversão concluída!")

Encontrados 3 arquivo(s) .txt para converter

Processando: combinada2025-09.txt
  ✓ Lido com sucesso (encoding: iso-8859-1)
  ✓ Dimensões: 193097.0 linhas x 90.0 colunas
  ✓ Convertido para: C:\Users\Casal-Inked\Documents\Jupyter_Lab\Dataset\ANAC\processed\combinada2025-09.parquet

Processando: combinada2025-10.txt
  ✗ Erro ao processar 'C:\Users\Casal-Inked\Documents\Jupyter_Lab\Dataset\ANAC\combinada2025-10.txt': Não foi possível ler o arquivo com nenhum encoding testado

Processando: combinada2025-11.txt
  ✓ Lido com sucesso (encoding: iso-8859-1)
  ✓ Dimensões: 193231.0 linhas x 90.0 colunas
  ✓ Convertido para: C:\Users\Casal-Inked\Documents\Jupyter_Lab\Dataset\ANAC\processed\combinada2025-11.parquet

Conversão concluída!
